I will be performing a survival analysis of the amount of days on Ibruntinib, where the event of interest is either:
- death 
- Major or minor bleeding
- Major bleeding
- (everything else is censored). 

I'll be applying the cox proportional hazards model. 

The relevant independent variables are: 
age of diagnosis, gender, platelet count, anti-coagulation, antiplatelet, invasive procedure (which I believe is PMHx bleeding risk), (maybe INR later), Molecular/cytogenetics, anemia, prior lines of therapy, major bleed (major vs. minor vs. none)

The relevant output variables are: duration on ibrunitinb and the event of interest 

Load necessary libraries 

In [1]:
library("survival")
library("survminer")

print("complete")

Loading required package: ggplot2
Loading required package: ggpubr


[1] "complete"


Load dataset

In [2]:
# Import the data and look at the first six rows
bleed_data <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/Bleeding_study/Ibrutinib Data Set, July 13,2021, de- identified data_cleaned_forSurvAnal.csv')
## bleed_data_cyto is the dataset with the rows that have 'unknown' cytogenetics removed. Only use this data set 
## when plotting survival curves where cytogenetics are the stratification factor 
bleed_data_cyto <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/Bleeding_study/Ibrutinib Data Set, July 13,2021, de- identified data_cleaned_forSurvAnal_CtyoCleaned.csv')
#head(bleed_data_cyto)

Univariate Cox regression analysis. Can adjust the event of interest, and the stratification factor. In this case, looking at when major bleeding is the event of interest, and Anemia (hb <100 (Y/N) is the stratification factor. 

In [3]:
res.cox <- coxph(Surv(Bleed.2_Time_on_Ibruntinib,Bleed.2_MajOnly.1...censored..2...event.) ~ Anemia..hb...100...Y.N., data =  bleed_data)
summary(res.cox)

Call:
coxph(formula = Surv(Bleed.2_Time_on_Ibruntinib, Bleed.2_MajOnly.1...censored..2...event.) ~ 
    Anemia..hb...100...Y.N., data = bleed_data)

  n= 168, number of events= 16 
   (1 observation deleted due to missingness)

                          coef exp(coef) se(coef)     z Pr(>|z|)  
Anemia..hb...100...Y.N. 1.3311    3.7853   0.5775 2.305   0.0212 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                        exp(coef) exp(-coef) lower .95 upper .95
Anemia..hb...100...Y.N.     3.785     0.2642     1.221     11.74

Concordance= 0.682  (se = 0.047 )
Likelihood ratio test= 6.26  on 1 df,   p=0.01
Wald test            = 5.31  on 1 df,   p=0.02
Score (logrank) test = 6.15  on 1 df,   p=0.01


Multivariate Cox regression analysis. Can adjust the event of interest, and the stratification factors.

In [4]:
# excluding HR molecular cytogenetics because then I would have to 
# eliminate all the rows with "unknown" cytogenetics 

res.cox <- coxph(Surv(Bleed.2_Time_on_Ibruntinib,Bleed.2_MajOnly.1...censored..2...event.) ~  Plt.Nadir.while.on.Ibrutinib + anticoagulation..Y.N. + Anemia..hb...100...Y.N., data = bleed_data)
summary(res.cox)

Call:
coxph(formula = Surv(Bleed.2_Time_on_Ibruntinib, Bleed.2_MajOnly.1...censored..2...event.) ~ 
    Plt.Nadir.while.on.Ibrutinib + anticoagulation..Y.N. + Anemia..hb...100...Y.N., 
    data = bleed_data)

  n= 168, number of events= 16 
   (1 observation deleted due to missingness)

                                  coef exp(coef)  se(coef)      z Pr(>|z|)  
Plt.Nadir.while.on.Ibrutinib -0.005031  0.994982  0.005606 -0.897   0.3696  
anticoagulation..Y.N.         1.257831  3.517783  0.524479  2.398   0.0165 *
Anemia..hb...100...Y.N.       1.378314  3.968204  0.608248  2.266   0.0234 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                             exp(coef) exp(-coef) lower .95 upper .95
Plt.Nadir.while.on.Ibrutinib     0.995     1.0050    0.9841     1.006
anticoagulation..Y.N.            3.518     0.2843    1.2584     9.833
Anemia..hb...100...Y.N.          3.968     0.2520    1.2046    13.072

Concordance= 0.773  (se = 0.061 )
Likelihood ratio test

Plot survival curves if I want? http://www.sthda.com/english/wiki/cox-proportional-hazards-model It explains how to do so here